# All pairs shortest path algorithm

Shortest paths between every pair of vertices -algorithm implemented by QUBO in quantum annealer. Classical version of this is Floyd-Warshall algorithm: https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm

Here directed graph $G=(V,E)$ has sets of vertices $V$ and edges $E \subseteq \{(x,y)|(x,y) \in V^2 and x \not = y\}$. For every edge there is weight $w_{xy}$. Task is to find path with minimum sum of weights for every combination of $(s,t) \in E^2$.

In this solution binary vertex $x$ to be solved has $|V|+|V|+|E|$ elements. First $|V|$ elements indicate the starting vertex and second $|V|$ elements indicate the ending vertex. Last $|E|$ elements would define which edges are part of the shortest path. 

Penalty $p=\sum w_{xy}+1$.

QUBO should implement following constraints to proper $x$: 
1. There must be exactly one starting vertex $s$ (if more than one: penalty of $2p$)
2. There must be exactly one terminating vertex $t$ (if more than one: penalty of $2p$)
3. Vertices $s$ and $t$ must different (if not: penalty of $p$)
4. There must edge starting from $s$ (gain of $-p$ for each) and no edge terminating to $s$ ($p$ for each)
5. There must edge ending to $t$ (gain of $-p$ for each) and no edge starting from $t$ ($p$ for each)
6. Two edges should not start/end to the same vertex, for example $s$ or $t$ (if so: penalty of $2p$) 
7. Two edges should form a chain (every edge gives penalty of $p$ to diagonal, but proper chain of two edges each gives gain of $-p$)
8. Path having lower weights should be prioritised (penalty of $w$ to each edge)

Proper path with correct $s$ and $t$ would give energy level of $w-p$ (constraint 7 brings minimum $p$ and constraint 4 and 5 both minimum $-p$). Because $p=\sum w_{xy}$ all samples with energy level below zero are correct paths. From this set of samples we choose lowest energy level sample for each $(s,t)$. 

In [1]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector
import networkx as nx
from networkx.classes.function import path_weight
import random

## Function to create QUBO

E is array of tuples (1st vertice, 2nd vertice, weight), veritices is number of vertices, p is penalty.

In [2]:
def create_qubo(E,vertices,p):
    edges = len(E)
    Q = np.zeros((2*vertices + edges, 2*vertices + edges))

    # Constraints 1 and 2
    for i in range(vertices):
        for j in range(i+1,vertices):
            Q[i,j]=2*p
            Q[vertices+i,j+vertices] = 2*p
        
    # Constraint 3
    for i in range(vertices):
        Q[i,i+vertices] = p

    # Constraint 4
    for v in range(vertices):
        for i,e in enumerate(E):
            if e[0]==v:
                Q[v,vertices*2+i] = -p
            if e[1]==v:
                Q[v,vertices*2+i] = p

    # Constraint 5
    for v in range(vertices):
        for i,e in enumerate(E):
            if e[1]==v:
                Q[vertices+v,vertices*2+i] = -p
            if e[0]==v:
                Q[vertices+v,vertices*2+i] = p

    # Constraint 6
    for i in range(edges):
        for j in range(i+1,edges):
            if E[i][0]==E[j][0] or E[i][1]==E[j][1]:
                Q[vertices*2+i,vertices*2+j] = 2*p

    # Constraint 7
    for i in range(edges):
        Q[vertices*2+i,vertices*2+i] += p
        for j in range(i+1,edges):
            if E[i][1]==E[j][0] or E[i][0]==E[j][1]:
                Q[vertices*2+i,vertices*2+j] += -p

    # Constraint 8 
    for i in range(edges):
        Q[vertices*2+i,vertices*2+i] += E[i][2]
    
    return Q

## Some helper functions for getting results from sampleset

In [3]:
def path_from_sample(sample,E,vertices):
    s = 0
    t = 0
    w = 0
    for v in range(vertices):
        if sample['s'+str(v)]==1:
            s = v
        if sample['t'+str(v)]==1:
            t = v
    i = s
    path = [i]
    while i!=t:
        for e in E:
            if e[0]==i and sample[str(e[0]) + '-' + str(e[1])]==1:
                i = e[1]
                path.append(i)
                w += e[2]
    return (str(s)+'-'+str(t),path,w)
        

def result_info(sampleset, E, vertices):
    res = {}
    for s in sampleset.filter(lambda s: s.energy<0):
        st, path, w = path_from_sample(s,E,vertices)
        if st not in res:
            res[st]=(path,w)
    return res

def check_result(G,sampleset,E,vertices, verbose=True):
    ok = 0
    s = 0
    res = result_info(sampleset,E,vertices)
    for i in range(vertices):
        for j in range(vertices):
            if i!=j:
                if nx.has_path(G,i,j):
                    s += 1
                    p1 = [p for p in nx.all_shortest_paths(G,i,j,weight='weight')]
                    w = path_weight(G,p1[0],'weight')
                    if str(i)+'-'+str(j) in res.keys():
                        p2 = res[str(i)+'-'+str(j)]
                        if (not p2[0] in p1) and w!=p2[1]:
                            if verbose:
                                print('Path: '+str(p2[0])+' ('+str(p2[1])+'): correct: '+str(p1)+' ('+str(w)+')')
                        else:
                            ok += 1
                    else:
                        if verbose:
                            print('Path suggested: '+str(i)+'-'+str(j)+' missing: correct: '+str(p1)+' ('+str(w)+')')
    print('{:.0f}% ok'.format((100*ok/s)))

def make_G(E,vertices):
    G = nx.DiGraph()
    G.add_nodes_from([0, vertices-1])
    for e in E:
        G.add_edge(e[0], e[1], weight=e[2])
    return(G)

## Simple graph

### Define graph

Input graph is array of tuples (1st vertice, 2nd vertice, weight)

In [4]:
E1 = np.array([(0, 2, 1), (2, 1, 2), (1, 3, 3), (3, 2, 4), (0, 1, 5), (3, 4, 3), (2, 4, 8)])
vertices1 = 5
G1 = make_G(E1,vertices1)

Above graph visualised:

![](graph4.png)

### Max_count and labels

In [12]:
max_count1 = 0
edges = len(E1)
for e in E1:
    max_count1 += e[2]
print('Max count:',max_count1)

labels1 = {}
for i in range(vertices1):
    labels1[i]='s'+str(i)
    labels1[vertices1+i]='t'+str(i)   
for i,e in enumerate(E1):
    labels1[vertices1*2+i] = str(e[0]) + '-' + str(e[1])

Max count: 26


### Create QUBO and BQM

In [13]:
t1 = time.time()
Q1 = create_qubo(E1,vertices1,max_count1+1)
qubo_time = (time.time()-t1)*1000
print('Time used for construction Q (ms): {:.3f}\n'.format(qubo_time))

t1 = time.time()
bqm1 = dimod.BinaryQuadraticModel(Q1, 'BINARY')
bqm_time = (time.time()-t1)*1000
bqm1 = bqm1.relabel_variables(labels1, inplace=False)

Time used for construction Q (ms): 0.328



### Local deterministic solver

In [14]:
t1 = time.time()
sampleset = dimod.ExactSolver().sample(bqm1)
det_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}\n'.format(det_time))
print(sampleset.filter(lambda s: s.energy<0))

Time used (ms): 199.940

   0-1 0-2 1-3 2-1 2-4 3-2 3-4 s0 s1 s2 s3 s4 t0 t1 t2 t3 t4 energy num_oc.
0    0   1   0   0   0   0   0  1  0  0  0  0  0  0  1  0  0  -26.0       1
2    0   0   0   1   0   0   0  0  0  1  0  0  0  1  0  0  0  -25.0       1
1    0   1   0   1   0   0   0  1  0  0  0  0  0  1  0  0  0  -24.0       1
5    0   0   1   0   0   0   0  0  1  0  0  0  0  0  0  1  0  -24.0       1
16   0   0   0   0   0   0   1  0  0  0  1  0  0  0  0  0  1  -24.0       1
8    0   0   0   0   0   1   0  0  0  0  1  0  0  0  1  0  0  -23.0       1
3    0   0   1   1   0   0   0  0  0  1  0  0  0  0  0  1  0  -22.0       1
11   1   0   0   0   0   0   0  1  0  0  0  0  0  1  0  0  0  -22.0       1
4    0   1   1   1   0   0   0  1  0  0  0  0  0  0  0  1  0  -21.0       1
7    0   0   0   1   0   1   0  0  0  0  1  0  0  1  0  0  0  -21.0       1
13   0   0   1   0   0   0   1  0  1  0  0  0  0  0  0  0  1  -21.0       1
6    0   0   1   0   0   1   0  0  1  0  0  0  0  0  1  0  0  -

In [15]:
for k,v in result_info(sampleset, E1, vertices1).items():
    print('Route '+k+': '+str(v[0])+', weight '+str(v[1]))

Route 0-2: [0, 2], weight 1
Route 2-1: [2, 1], weight 2
Route 0-1: [0, 2, 1], weight 3
Route 1-3: [1, 3], weight 3
Route 3-4: [3, 4], weight 3
Route 3-2: [3, 2], weight 4
Route 2-3: [2, 1, 3], weight 5
Route 0-3: [0, 2, 1, 3], weight 6
Route 3-1: [3, 2, 1], weight 6
Route 1-4: [1, 3, 4], weight 6
Route 1-2: [1, 3, 2], weight 7
Route 2-4: [2, 1, 3, 4], weight 8
Route 0-4: [0, 2, 1, 3, 4], weight 9


In [16]:
check_result(G1,sampleset,E1,vertices1)

100% ok


### Local heuristic classical solver

In [17]:
num_reads = 5000
t1 = time.time()
sampleset2 = SimulatedAnnealingSampler().sample(bqm1, num_reads=num_reads).aggregate()
heur_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}\n'.format(heur_time))
print(sampleset2.filter(lambda s: s.energy<0))

Time used (ms): 1598.380

   0-1 0-2 1-3 2-1 2-4 3-2 3-4 s0 s1 s2 s3 s4 t0 t1 t2 t3 t4 energy num_oc.
2    0   1   0   0   0   0   0  1  0  0  0  0  0  0  1  0  0  -26.0     646
3    0   0   0   1   0   0   0  0  0  1  0  0  0  1  0  0  0  -25.0     581
0    0   1   0   1   0   0   0  1  0  0  0  0  0  1  0  0  0  -24.0     418
6    0   0   1   0   0   0   0  0  1  0  0  0  0  0  0  1  0  -24.0     375
14   0   0   0   0   0   0   1  0  0  0  1  0  0  0  0  0  1  -24.0     401
5    0   0   0   0   0   1   0  0  0  0  1  0  0  0  1  0  0  -23.0     370
9    0   0   1   1   0   0   0  0  0  1  0  0  0  0  0  1  0  -22.0     254
11   1   0   0   0   0   0   0  1  0  0  0  0  0  1  0  0  0  -22.0     267
1    0   1   1   1   0   0   0  1  0  0  0  0  0  0  0  1  0  -21.0     209
4    0   0   0   1   0   1   0  0  0  0  1  0  0  1  0  0  0  -21.0     230
13   0   0   1   0   0   0   1  0  1  0  0  0  0  0  0  0  1  -21.0     260
15   0   0   1   0   0   1   0  0  1  0  0  0  0  0  1  0  0  

In [18]:
for k,v in result_info(sampleset2,E1,vertices1).items():
    print('Route '+k+': '+str(v[0])+', weight '+str(v[1]))

Route 0-2: [0, 2], weight 1
Route 2-1: [2, 1], weight 2
Route 0-1: [0, 2, 1], weight 3
Route 1-3: [1, 3], weight 3
Route 3-4: [3, 4], weight 3
Route 3-2: [3, 2], weight 4
Route 2-3: [2, 1, 3], weight 5
Route 0-3: [0, 2, 1, 3], weight 6
Route 3-1: [3, 2, 1], weight 6
Route 1-4: [1, 3, 4], weight 6
Route 1-2: [1, 3, 2], weight 7
Route 2-4: [2, 4], weight 8
Route 0-4: [0, 2, 1, 3, 4], weight 9


In [19]:
check_result(G1,sampleset2,E1,vertices1)

100% ok


### Quantum solver

In [20]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [28]:
num_reads = 2000
sampleset3 = EmbeddingComposite(machine).sample(bqm1, num_reads=num_reads).aggregate()

In [29]:
qpu_time = sampleset3.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset3.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Physical qubits used: {}\n'.format(qubits))
print(sampleset3.filter(lambda s: s.energy<0))

QPU time used (ms): 222.2
Physical qubits used: 32

   0-1 0-2 1-3 2-1 2-4 3-2 3-4 s0 s1 s2 s3 s4 t0 t1 t2 t3 t4 energy num_oc. ...
0    0   1   0   0   0   0   0  1  0  0  0  0  0  0  1  0  0  -26.0      47 ...
1    0   0   0   1   0   0   0  0  0  1  0  0  0  1  0  0  0  -25.0      29 ...
2    0   0   0   0   0   0   1  0  0  0  1  0  0  0  0  0  1  -24.0      19 ...
3    0   0   1   0   0   0   0  0  1  0  0  0  0  0  0  1  0  -24.0      32 ...
4    0   1   0   1   0   0   0  1  0  0  0  0  0  1  0  0  0  -24.0      40 ...
5    0   0   0   0   0   1   0  0  0  0  1  0  0  0  1  0  0  -23.0      51 ...
6    0   0   1   1   0   0   0  0  0  1  0  0  0  0  0  1  0  -22.0      22 ...
7    1   0   0   0   0   0   0  1  0  0  0  0  0  1  0  0  0  -22.0      30 ...
8    0   0   1   0   0   0   1  0  1  0  0  0  0  0  0  0  1  -21.0      20 ...
9    0   1   1   1   0   0   0  1  0  0  0  0  0  0  0  1  0  -21.0      19 ...
10   0   0   0   1   0   1   0  0  0  0  1  0  0  1  0  0  0  -21.0 

In [30]:
for k,v in result_info(sampleset3,E1,vertices1).items():
    print('Route '+k+': '+str(v[0])+', weight '+str(v[1]))

Route 0-2: [0, 2], weight 1
Route 2-1: [2, 1], weight 2
Route 3-4: [3, 4], weight 3
Route 1-3: [1, 3], weight 3
Route 0-1: [0, 2, 1], weight 3
Route 3-2: [3, 2], weight 4
Route 2-3: [2, 1, 3], weight 5
Route 0-3: [0, 2, 1, 3], weight 6
Route 3-1: [3, 2, 1], weight 6
Route 1-4: [1, 3, 4], weight 6
Route 1-2: [1, 3, 2], weight 7
Route 2-4: [2, 4], weight 8
Route 0-4: [0, 2, 4], weight 9


In [31]:
check_result(G1,sampleset3,E1,vertices1)

100% ok


### Timings

In [18]:
print('Construting QUBO: {:.3f}'.format(qubo_time))
print('Construting BQM: {:.3f}'.format(bqm_time))
print('\nLocal deterministic solver: {:.1f}'.format(det_time))
print('Local heuristic solver: {:.1f}'.format(heur_time))
print('Quantum solver: {:.1f}'.format(qpu_time))

Construting QUBO: 0.449
Construting BQM: 3.993

Local deterministic solver: 256.8
Local heuristic solver: 1580.4
Quantum solver: 82.5


## More complex graph

In [32]:
seed = 42
vertices2 = 10
random.seed(seed)
G2 = nx.gnp_random_graph(vertices2, 0.30, seed, directed=True)
nx.set_edge_attributes(G2, {e: {'weight': random.randint(1, 10)} for e in G2.edges})

In [33]:
E2 = [] 
for e in G2.edges(data=True):
    E2.append((e[0],e[1],e[2]['weight']))
print('Number of edges:',len(E2))
print('Number of vertices:',vertices2)

max_count2 = 0
for e in E2:
    max_count2 += e[2]
print('Max count:',max_count2)

labels2 = {}
for i in range(vertices2):
    labels2[i]='s'+str(i)
    labels2[vertices2+i]='t'+str(i)   
for i,e in enumerate(E2):
    labels2[vertices2*2+i] = str(e[0]) + '-' + str(e[1])

Number of edges: 34
Number of vertices: 10
Max count: 166


### QUBO and BQM

In [34]:
t1 = time.time()
p = max_count2
Q2 = create_qubo(E2,vertices2,p)
qubo_time = (time.time()-t1) * 1000
print('Time used for construction Q (ms): {:.3f}'.format(qubo_time))

t1 = time.time()
bqm2 = dimod.BinaryQuadraticModel(Q2, 'BINARY')
bqm_time = (time.time()-t1) * 1000
bqm2 = bqm2.relabel_variables(labels2, inplace=False)
print('Time used for construction BQM (ms): {:.3f}'.format(bqm_time))

print('\nNumber of logical qubits needed:',Q2.shape[0])
print('Number of couplers needed:', len(bqm2.quadratic))

Time used for construction Q (ms): 0.790
Time used for construction BQM (ms): 0.320

Number of logical qubits needed: 54
Number of couplers needed: 438


### Local heuristic solver

In [35]:
num_reads = 10000
t1 = time.time()
sampleset4 = SimulatedAnnealingSampler().sample(bqm2, num_reads=num_reads).aggregate()
heur_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}'.format(heur_time))
print('Lowest energy', sampleset4.first.energy)

Time used (ms): 7585.177
Lowest energy -165.0


In [36]:
check_result(G2,sampleset4,E2,vertices2)

Path: [8, 7, 3, 0, 4, 6, 2, 1, 5] (49): correct: [[8, 6, 2, 1, 5]] (29)
99% ok


### Quantum solver

In [37]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [38]:
num_reads = 2000
sampleset5 = EmbeddingComposite(machine).sample(bqm2, num_reads=num_reads)

In [41]:
qpu_time = sampleset5.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset5.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Lowest energy', sampleset5.first.energy)
print('Physical qubits used: {}'.format(qubits))

QPU time used (ms): 343.7
Lowest energy -163.0
Physical qubits used: 254


In [40]:
check_result(G2,sampleset5,E2,vertices2,verbose=False)
print('Chain breaks:',sampleset5.first.chain_break_fraction)

3% ok
Chain breaks: 0.0


In [44]:
print(sampleset5)

     0-2 0-3 0-4 0-8 1-0 1-2 1-4 1-5 1-8 2-1 2-6 2-9 ... t9 energy num_oc. ...
0      0   0   0   0   0   0   0   0   0   0   0   0 ...  0 -163.0       1 ...
339    0   0   0   0   0   0   1   0   0   0   0   1 ...  0 -140.0       1 ...
335    0   0   1   0   0   0   0   0   1   0   0   1 ...  0 -136.0       1 ...
13     1   0   0   0   0   0   0   1   0   0   0   1 ...  0 -128.0       1 ...
134    0   0   0   0   0   0   0   0   0   0   0   0 ...  0    5.0       1 ...
2      0   0   0   0   0   0   0   0   0   0   0   0 ...  0    8.0       1 ...
4      0   1   0   0   0   1   0   0   0   0   0   1 ...  0   12.0       1 ...
248    0   0   0   0   0   1   0   0   0   0   0   1 ...  0   12.0       1 ...
318    0   1   0   0   1   0   0   0   0   0   0   1 ...  0   13.0       1 ...
5      0   1   0   0   0   0   0   0   0   0   0   0 ...  0   17.0       1 ...
272    1   0   0   0   0   0   0   1   0   0   0   1 ...  1   17.0       1 ...
6      1   0   0   0   0   0   0   0   0   0   0   1

In [61]:
chain_strength = p*2+1 #Default 1
num_reads = 1000
sampleset5 = EmbeddingComposite(machine).sample(bqm2, num_reads=num_reads, chain_strength=chain_strength)
qpu_time = sampleset5.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset5.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Lowest energy', sampleset5.first.energy)
print('Chain breaks:',sampleset5.first.chain_break_fraction)
print('Physical qubits used: {}\n'.format(qubits))
check_result(G2,sampleset5,E2,vertices2,verbose=False)

QPU time used (ms): 153.7
Lowest energy -19.0
Chain breaks: 0.018518518518518517
Physical qubits used: 232

1% ok


### Timings

In [40]:
print('Construting QUBO (ms): {:.3f}'.format(qubo_time))
print('Construting BQM (ms): {:.3f}'.format(bqm_time))
print('\nLocal heuristic solver (ms): {:.1f}'.format(heur_time))
print('Quantum solver (ms): {:.1f}'.format(qpu_time))

Construting QUBO (ms): 0.769
Construting BQM (ms): 0.353

Local heuristic solver (ms): 7592.9
Quantum solver (ms): 294.0


## Test procedure #1

Test with 20 random graphs, with local heuristic solver

In [41]:
for seed in range(20):
    vertices = 10
    random.seed(seed)
    G = nx.gnp_random_graph(vertices, 0.30, seed, directed=True)
    nx.set_edge_attributes(G, {e: {'weight': random.randint(1, 10)} for e in G.edges})
    
    E = [] 
    for e in G.edges(data=True):
        E.append((e[0],e[1],e[2]['weight']))

    max_count = 0
    for e in E:
        max_count += e[2]

    labels = {}
    for i in range(vertices):
        labels[i]='s'+str(i)
        labels[vertices+i]='t'+str(i)   
    for i,e in enumerate(E):
        labels[vertices*2+i] = str(e[0]) + '-' + str(e[1])
        
    
    Q = create_qubo(E,vertices,max_count)
    bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
    bqm = bqm.relabel_variables(labels, inplace=False)

    num_reads = 20000
    sampleset = SimulatedAnnealingSampler().sample(bqm, num_reads=num_reads).aggregate()
    print('\nSeed:',seed)
    check_result(G,sampleset,E,vertices)


Seed: 0
100% ok

Seed: 1
100% ok

Seed: 2
100% ok

Seed: 3
100% ok

Seed: 4
100% ok

Seed: 5
100% ok

Seed: 6
100% ok

Seed: 7
100% ok

Seed: 8
100% ok

Seed: 9
100% ok

Seed: 10
100% ok

Seed: 11
100% ok

Seed: 12
100% ok

Seed: 13
100% ok

Seed: 14
100% ok

Seed: 15
100% ok

Seed: 16
100% ok

Seed: 17
100% ok

Seed: 18
100% ok

Seed: 19
Path: [0, 7, 9, 4, 3] (10): correct: [[0, 9, 4, 3]] (9)
Path: [2, 0, 7, 9, 4, 6, 5, 1] (24): correct: [[2, 0, 9, 4, 6, 5, 1]] (23)
98% ok
